In [11]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [3]:
throws_df = pd.read_csv('../data/raw/all_games_0704.csv')
non_bricks = throws_df[(throws_df['thrower_x'] != 0) & (throws_df['thrower_y'] != 40)]
non_bricks_clean = non_bricks.dropna(subset=['thrower_x', 'thrower_y'])

/var/folders/r4/cqt7s71n4k18qp94swd6_njr0000gn/T/ipykernel_33504/1692646384.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  throws_df = pd.read_csv('../data/raw/all_games_0704.csv')


In [25]:
def get_bin_data(df, nbinsx, nbinsy):
    hist, xedges, yedges = np.histogram2d(df['thrower_x'], df['thrower_y'], bins=[nbinsx, nbinsy])
    x_coords = []
    y_coords = []
    counts = []
    for i in range(nbinsx):
        for j in range(nbinsy):
            x = (xedges[i] + xedges[i+1]) / 2
            y = (yedges[j] + yedges[j+1]) / 2
            count = hist[i][j]
            x_coords.append(x)
            y_coords.append(y)
            counts.append(count)
    return x_coords, y_coords, counts


def shot_plot(teamID, nbinsx=10, nbinsy=15):
    shots = throws_df[(throws_df.gameID.apply(lambda x:x[:4]) == '2023') & (throws_df['thrower_x'] != 0) & (throws_df['thrower_y'] != 40) & ((throws_df['home_teamID'] == teamID) | (throws_df['away_teamID'] == teamID))].dropna(subset=['thrower_x', 'thrower_y'])
    x_coords, y_coords, counts = get_bin_data(shots, nbinsx, nbinsy)
    # Create the figure
    fig = go.Figure()

    # Add the main scatter plot
    fig.add_trace(go.Scatter(
        x=x_coords, y=y_coords, mode='markers', name='markers',
        marker=dict(
            size=counts, sizemode='area', sizeref = (2. * max(counts) / ((500/nbinsx) ** 2)), sizemin=2.5,
            color=counts,
            line=dict(width=1, color='#333333'), symbol='circle',
        ),
        hovertemplate='Count: %{text}<extra></extra>',
        text=[int(x) for x in counts]
    ))

    # Add black horizontal lines at y = 20 and y = 100
    fig.add_shape(type='line', x0=-27, y0=20, x1=27, y1=20, line=dict(color='black', width=1))
    fig.add_shape(type='line', x0=-27, y0=100, x1=27, y1=100, line=dict(color='black', width=1))

    # Set the layout properties
    fig.update_layout(
        xaxis=dict(range=[-27, 27]),
        yaxis=dict(range=[0, 120]),
        showlegend=False,
        width=450,
        height=600,
        margin=dict(t=50, b=50, l=50, r=50),
    )
    

    # Show the plot
    fig.show()


In [22]:
np.array(counts).reshape(nbinsx, nbinsy).sum(1).shape

(10,)

In [65]:
teamID = 'phoenix'
nbinsx=10
nbinsy=15
shots = throws_df[(throws_df.gameID.apply(lambda x:x[:4]) == '2023') & (throws_df['thrower_x'] != 0) & (throws_df['thrower_y'] != 40) & ((throws_df['home_teamID'] == teamID) | (throws_df['away_teamID'] == teamID))].dropna(subset=['thrower_x', 'thrower_y'])
x_coords, y_coords, counts = get_bin_data(shots, nbinsx, nbinsy)
# Create the figure
fig = make_subplots(rows=2, cols=2,
                row_heights=[0.2, 0.8],
                column_widths=[0.8, 0.2],
                vertical_spacing = 0.02,
                horizontal_spacing = 0.02,
                shared_yaxes=True,
                shared_xaxes=False)

# Add the bar chart
fig.add_trace(go.Violin(
    x=shots.thrower_x,
    name='',
), row=1, col=1)

fig.add_trace(go.Violin(
    y=shots.thrower_y,
    name='',
    hovertemplate='check: %{y}<extra></extra>'
), row=2, col=2)

# Add the main scatter plot
fig.add_trace(go.Scatter(
    x=x_coords, y=y_coords, mode='markers', name='markers',
    marker=dict(
        size=counts, sizemode='area', sizeref = (2. * max(counts) / ((500/nbinsx) ** 2)), sizemin=2.5,
        color=counts,
        line=dict(width=1, color='#333333'), symbol='circle',
    ),
    hovertemplate='Count: %{text}<extra></extra>',
    text=[int(x) for x in counts]
), row = 2, col = 1)

# Add black horizontal lines at y = 20 and y = 100
fig.add_shape(type='line', x0=-27, y0=20, x1=27, y1=20, line=dict(color='black', width=1), row=2, col=1)
fig.add_shape(type='line', x0=-27, y0=100, x1=27, y1=100, line=dict(color='black', width=1), row=2, col=1)

fig.update_layout(
    xaxis=dict(range=[0, None]),  # Apply x-axis range to the scatter plot
    yaxis=dict(range=[0, None]),  # Apply y-axis range to the scatter plot
    xaxis2=dict(range=[-27, 27]),
    yaxis2=dict(range=[0, 120]),  
    showlegend=False,
    width=600,
    height=600,
    margin=dict(t=50, b=50, l=50, r=50)
)


# Show the plot
fig.show()


In [47]:
fig = px.violin(y=shots.thrower_y, box=True, # draw box plot inside the violin
                points='all', # can be 'outliers', or False
               )
fig.show()

In [26]:
shot_plot('phoenix', 15, 20)

In [7]:
shot_plot('mechanix', 15, 20)

In [167]:
shot_plot('empire', 15, 20)

In [168]:
shot_plot('shred', 15, 20)

In [156]:
shot_plot('breeze', 15, 20)

In [105]:
import plotly.graph_objects as go

# Create the figure
fig = go.Figure()

# Add the main scatter plot
fig.add_trace(go.Scatter(
    x=x_coords, y=y_coords, mode='markers', name='markers',
    marker=dict(
        size=counts, sizemode='area', sizeref=3, sizemin=2.5,
        color=counts,
        line=dict(width=1, color='#333333'), symbol='circle',
    ),
))

# Add black horizontal lines at y = 20 and y = 100
fig.add_shape(type='line', x0=-27, y0=20, x1=27, y1=20, line=dict(color='black', width=1))
fig.add_shape(type='line', x0=-27, y0=100, x1=27, y1=100, line=dict(color='black', width=1))

# Set the layout properties
fig.update_layout(
    xaxis=dict(range=[-27, 27]),
    yaxis=dict(range=[0, 120]),
    showlegend=False,
    width=600,
    height=600,
    margin=dict(t=50, b=50, l=50, r=50),
)

# Show the plot
fig.show()
